In [152]:
from pathlib import Path

import networkx as nx
import pandas as pd

SOURCE_PATH = Path('../bfs_source_data/')
TARGET_PATH = Path('../processed_data/')
INIT_GMDE_FILE = 'Gemeindestand_31_05_1981.xlsx'
MUTATION_FILE = 'Mutationen_1981_2023.xlsx'

In [153]:
def process_column_names(df, suffix=False):
    """Function to process the column names of a DataFrame

    Parameters
    ----------
    df: pd.DataFrame
    suffix: bool
        if True, adds (_old, _new) to columns of the same name

    Returns
    -------
    pd.DataFrame
    """

    if 'Mutationsnummer' in df.columns:
        df = df.drop(columns='Mutationsnummer')

    if suffix:
        new_col_names = [c + '_old' if '.1' not in c else c[:-2] + '_new'
                         for c in df.columns]
        new_col_names[-1] = new_col_names[-1].replace('_old', '')
        df.columns = new_col_names

    df.columns = [c.lower().replace(' ', '_').replace('-', '_')
                  for c in df.columns]

    return df


def create_date_dictionary(date_list):
    """Function to create a dictionary to map the date to the corresponding
    Gemeindestand, e.g. {'1981-01-01': '81_a'}

    Parameters
    ----------
    date_list: list
        List of dates in the form of ´YYYY-MM-DD´

    Returns
    -------
    dict
    """
    date_dict = {}
    year_suffix = {}

    for date_str in date_list:
        date_parts = date_str.split('-')
        year = date_parts[0]

        if year not in year_suffix:
            year_suffix[year] = 'a'
        else:
            year_suffix[year] = chr(ord(year_suffix[year]) + 1)

        suffix = year_suffix[year]
        date_dict[date_str] = f"{year[2:4]}_{suffix}"

    return date_dict


def build_graphs(initial_list, change_matrices):
    """Function to create a graph for each municipality
    to represent changes such as merging, splitting, and
    renaming of municipalities.

    Parameters
    ----------
    initial_list: list
        List of municipality numbers
    change_matrices: Union[List | Tuples]
        List of edges per Gemeindestand

    Returns
    -------
    Union[dict | nx.DiGraph]
    """
    # Create a dictionary to store the trees
    graphs = {}

    # Iterate over each element in the initial list
    for root in initial_list:
        # Create a directed graph
        G = nx.DiGraph()
        G.add_node(root)

        # Iterate through the change matrices to build the graph
        for edges in change_matrices:
            for edge in edges:
                original_value, new_value = edge
                if original_value in G:
                    G.add_node(new_value)
                    G.add_edge(original_value, new_value)
                    if (G.has_edge(original_value, original_value) & ((original_value, original_value) not in edges)):
                        G.remove_edge(original_value, original_value)

        # Store the graph in the dictionary
        graphs[root] = G

    return graphs

In [154]:
# read data
gmde_df = pd.read_excel(SOURCE_PATH / INIT_GMDE_FILE,
                        engine='openpyxl')
mutation_df = pd.read_excel(SOURCE_PATH / MUTATION_FILE, header=1,
                            engine='openpyxl')

# reformat column names and remove unnecessary ones
gmde_df = process_column_names(gmde_df)
gmde_df = gmde_df.drop(columns=['hist._nummer',
                                'datum_der_aufnahme',
                                'bezirksname'])
mutation_df = process_column_names(mutation_df, suffix=True)

# add column with Gemeindestand
dates = mutation_df.datum_der_aufnahme.unique()
mapping = create_date_dictionary(dates)
mutation_df['gemeindestand'] = (
    mutation_df['datum_der_aufnahme'].replace(mapping)
)

/Users/manuelbolz/opt/anaconda3/envs/py_env/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/manuelbolz/opt/anaconda3/envs/py_env/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [ ]:
# read data
gmde_df = pd.read_excel(SOURCE_PATH / INIT_GMDE_FILE,
                        engine='openpyxl')
mutation_df = pd.read_excel(SOURCE_PATH / MUTATION_FILE, header=1,
                            engine='openpyxl')

# reformat column names and remove unnecessary ones
gmde_df = process_column_names(gmde_df)
gmde_df = gmde_df.drop(columns=['hist._nummer',
                                'datum_der_aufnahme',
                                'bezirksname'])
mutation_df = process_column_names(mutation_df, suffix=True)

# add column with Gemeindestand
mutation_df= mutation_df.rename(columns={'datum_der_aufnahme': 'gemeindestand'})

In [155]:
mutation_df

,kanton_old,bezirks_nummer_old,bfs_gde_nummer_old,gemeindename_old,kanton_new,bezirks_nummer_new,bfs_gde_nummer_new,gemeindename_new,datum_der_aufnahme,gemeindestand
0,VD,2219,5938,Yverdon,VD,2219,5938,Yverdon-les-Bains,1982-01-01,82_a
1,FR,1004,2203,Lossy,FR,1004,2203,Lossy-Formangueires,1982-01-01,82_a
2,FR,1004,2195,Formangueires,FR,1004,2203,Lossy-Formangueires,1982-01-01,82_a
3,GR,1809,3809,Santa Domenica,GR,1809,3808,Rossa,1982-02-25,82_b
4,GR,1809,3802,Augio,GR,1809,3808,Rossa,1982-02-25,82_b
...,...,...,...,...,...,...,...,...,...,...
3097,AG,1905,4139,Menziken,AG,1905,4139,Menziken,2023-01-01,23_a
3098,AG,1906,4179,Ueken,AG,1906,4186,Herznach-Ueken,2023-01-01,23_a
3099,AG,1906,4166,Herznach,AG,1906,4186,Herznach-Ueken,2023-01-01,23_a
3100,JU,2603,6787,Damphreux,JU,2603,6811,Damphreux-Lugnez,2023-01-01,23_a


In [220]:
# GEMEINDESTAND-CREATOR

l_init = gmde_df['bfs_gde_nummer'].to_list() # Data 01.06.1981

change_matrices = []
for gemeindestand in mutation_df.gemeindestand.unique():
    temp_df = mutation_df.query("gemeindestand == @gemeindestand")
    edges = list(zip(temp_df.bfs_gde_nummer_old, temp_df.bfs_gde_nummer_new))
    change_matrices.append(edges)
    
for i, gemeindestand in enumerate(mutation_df.gemeindestand.unique()):
    graphs = build_graphs(l_init, change_matrices[:i+1])
    gemeinde_mapping = {}
    for root, graph in graphs.items():
        leaves = (
            [node for node, out_degree in graph.out_degree() if out_degree == 0] 
            + [node for node in list(nx.nodes_with_selfloops(graph))]
        )
        gemeinde_mapping[root] = leaves

    col1 = []
    for values in gemeinde_mapping.values():
        for v in values:
            if v not in col1:
                col1.append(v)

    result = pd.DataFrame({gemeindestand: col1})
    result.to_excel(TARGET_PATH / f'Gemeindestand_{gemeindestand}.xlsx', index=False)


# GEMEINDEMAPPING-TABLE

col1 = []
col2 = []
for key, value in gemeinde_mapping.items():
    for v in value:
        col1.append(key)
        col2.append(v)

result = pd.DataFrame({'82_a': col1,
                       '23_a': col2})
result.to_excel(TARGET_PATH / 'Gemeindemapping_82a_23a.xlsx', index=False)

In [ ]:
# GEMEINDESTAND-CREATOR

l_init = gmde_df['bfs_gde_nummer'].to_list() # Data 01.06.1981

change_matrices = []
for gemeindestand in mutation_df.gemeindestand.unique():
    temp_df = mutation_df.query("gemeindestand == @gemeindestand")
    edges = list(zip(temp_df.bfs_gde_nummer_old, temp_df.bfs_gde_nummer_new))
    change_matrices.append(edges)
    
for i, gemeindestand in enumerate(mutation_df.gemeindestand.unique()):
    graphs = build_graphs(l_init, change_matrices[:i+1])
    gemeinde_mapping = {}
    for root, graph in graphs.items():
        leaves = (
            [node for node, out_degree in graph.out_degree() if out_degree == 0] 
            + [node for node in list(nx.nodes_with_selfloops(graph))]
        )
        gemeinde_mapping[root] = leaves

    col1 = []
    for values in gemeinde_mapping.values():
        for v in values:
            if v not in col1:
                col1.append(v)

    result = pd.DataFrame({gemeindestand: col1})
    result.to_excel(TARGET_PATH / f'Gemeindestand_{gemeindestand}.xlsx', index=False)


# GEMEINDEMAPPING-TABLE

col1 = []
col2 = []
for key, value in gemeinde_mapping.items():
    for v in value:
        col1.append(key)
        col2.append(v)

result = pd.DataFrame({'82_a': col1,
                       '23_a': col2})
result.to_excel(TARGET_PATH / 'Gemeindemapping_82a_23a.xlsx', index=False)

In [149]:
from datetime import date
today = date.today().isoformat()

In [151]:
today

'2023-09-07'

In [208]:
# TEST
gmde_list = [el for el in gemeinde_mapping.values()]
gmde_list = []
for val in gemeinde_mapping.values():
    gmde_list += val
len(set(gmde_list))

In [156]:
directory = TARGET_PATH / 'gmde_stde'
df = pd.DataFrame({'gemeindestand': [],
                   'anz_gmde':[]})
for file in directory.glob('*'):
    df_temp = pd.read_excel(file)
    df.loc[len(df)] = [df_temp.columns.values[0], df_temp.iloc[:, 0].nunique()]

In [157]:
df = df.sort_values(['gemeindestand']).reset_index(drop=True)
df.to_excel(TARGET_PATH / 'Anzahl_Gemeinden_pro_Gemeindestand.xlsx', index=False)

In [143]:
df

,gmde_stand,anz_gmde
0,09_c,2624
1,21_c,2163
2,10_a,2596
3,06_f,2726
4,02_a,2865
...,...,...
80,19_a,2212
81,10_b,2588
82,15_a,2324
83,06_e,2726


In [111]:
df = pd.read_excel(SOURCE_PATH / "Gleiche Rechte für Mann und Frau.xlsx", header=2, nrows=2396)

In [112]:
col_names = ['82_a', 'gmde_name_82_a'] + df.columns[4:].to_list()
df = df.drop(columns=['Unnamed: 2', 'Unnamed: 3'])
df.columns = col_names

In [113]:
df = df[df.gmde_name_82_a.str.contains('...', regex=False)]
df = df[df['82_a'].astype(int) < 7000]

In [115]:
df['82_a'] = [int(num.lstrip('0')) for num in df['82_a']]

In [116]:
df['gmde_name_82_a'] = [name.strip('.') for name in df.gmde_name_82_a]

In [117]:
df['82_a'].nunique()

2145

In [94]:
mapping = pd.read_excel(TARGET_PATH / 'Gemeindemapping_82a_23a.xlsx')
gmde_names = pd.read_excel(SOURCE_PATH / 'Gemeindestand_01_01_2023.xlsx')

/Users/manuelbolz/opt/anaconda3/envs/py_env/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [96]:
mapping = mapping.merge(gmde_names[['BFS Gde-nummer', 'Gemeindename']], how='left', left_on='23_a', right_on='BFS Gde-nummer')

In [98]:
mapping = (
    mapping
    .drop(columns='BFS Gde-nummer')
    .rename(columns={'Gemeindename': 'gmde_name_23_a'})
)

In [100]:
df = df.merge(mapping, how='left', on='82_a')

In [101]:
df

,82_a,gmde_name_82_a,Stimmberechtigte,Abgegebene Stimmen,Beteiligung in %,Gültige Stimmzettel,Ja,Nein,Ja in %,23_a,gmde_name_23_a
0,1,Aeugst am Albis,577,285,49.39,282,181,101,64.18,1.0,Aeugst am Albis
1,2,Affoltern am Albis,4543,1557,34.27,1536,941,595,61.26,2.0,Affoltern am Albis
2,3,Bonstetten,1411,581,41.18,574,401,173,69.86,3.0,Bonstetten
3,4,Hausen am Albis,1330,450,33.83,440,284,156,64.55,4.0,Hausen am Albis
4,5,Hedingen,1235,452,36.6,448,297,151,66.29,5.0,Hedingen
...,...,...,...,...,...,...,...,...,...,...,...
2202,6806,Vendlincourt,349,77,22.06,75,43,32,57.33,6806.0,Vendlincourt
2203,6807,Basse-Allaine,994,186,18.71,185,136,49,73.51,NaN,NaN
2204,6808,Clos du Doubs,1012,254,25.1,248,171,77,68.95,NaN,NaN
2205,6809,Haute-Ajoie,825,178,21.58,178,147,31,82.58,NaN,NaN


In [102]:
mapping

,82_a,23_a,gmde_name_23_a
0,1,1,Aeugst am Albis
1,2,2,Affoltern am Albis
2,3,3,Bonstetten
3,4,4,Hausen am Albis
4,5,5,Hedingen
...,...,...,...
3082,6802,6809,Haute-Ajoie
3083,6803,6809,Haute-Ajoie
3084,6804,6808,Clos du Doubs
3085,6805,6808,Clos du Doubs
